# 【 プログラミング入門 ～ Pandas ～ 】　勉強会解答例　  by にんにん

SSDSE-E-2023のデータフレームdfの構成は以下の通りです。
* 「項目情報」は、columnsの「項目コード」と0行目の「項目年次」、および1行目の「項目名」です。
* 「地域情報」は、0列目の「地域コード」と1列目の「都道府県」です。
 
SSDSE-E-2023のデータフレームdfは、「項目年次」は持つものの時系列データではありません。
* 「項目コード」は「項目名」と「地域コード」は「都道府県」と対応していることから、ほぼ地域と項目の2次元データと見做せますが、ここでは3次元以上のデータを扱う際に有用なMultiIndexの練習素材として取り扱います。

In [1]:
import pandas as pd
df = pd.read_excel('.\\230501\\SSDSE-E-2023.xlsx')
df.head()

,SSDSE-E-2023,Prefecture,A1101,A1102,A1301,A1302,A1303,A1700,A4101,A4103,...,I5103,I6100,I6200,I6300,J2503,J2506,L3221,L322101,L322102,L322109
0,NaN,年度,2021,2021,2021,2021,2021,2020,2020,2019,...,2020,2020,2020,2020,2020,2020,2021,2021,2021,2021
1,地域コード,都道府県,総人口,日本人人口,15歳未満人口,15～64歳人口,65歳以上人口,外国人人口,出生数,合計特殊出生率,...,歯科診療所数,医師数,歯科医師数,薬剤師数,保育所等数,保育所等在所児数,消費支出（二人以上の世帯）,食料費（二人以上の世帯）,住居費（二人以上の世帯）,教養娯楽費（二人以上の世帯）
2,R00000,全国,125502000,122780000,14784000,74504000,36214000,2402460,840835,1.36,...,67874,339623,107443,321982,29474,2624335,279024,75761,18329,24545
3,R01000,北海道,5183000,5147000,544000,2953000,1686000,34321,29523,1.24,...,2840,13731,4418,11802,1045,79404,268396,71448,22013,23762
4,R02000,青森県,1221000,1216000,127000,676000,419000,5409,6837,1.38,...,511,2773,735,2345,475,30723,245117,69503,13232,19092


## 解答 1

* MultiIndexは、set_indexメソッドでindexを追加することもできますが、出来上がりのデータ構造がイメージできているときは明示的に設定する方が効率的です。
* MultiIndexはindexやcolumnsの要素がtupleになるため操作にひと手間かかりますが、一方、コーディングの工夫で効率化も可能です。
* 特に今回のデータセットはデータ内にindexやcolumnsの要素が混在しているため、明示的にMultiIndex化して整理した方が各データの意味が明確になります。

In [2]:
# 明示的なMultiIndexの設定
df1 = df

# MultiIndexのcolumns,index各々のlevelの取得
level_c0 = df1.columns
level_c1 = df1.iloc[0]
level_c2 = df1.iloc[1]
level_i0 = df1.iloc[:,0]
level_i1 = df1.iloc[:,1]

# Multiindexの作成とcolumns属性、index属性への代入
multi_index_c = pd.MultiIndex.from_tuples(zip(level_c0, level_c1, level_c2))
multi_index_i = pd.MultiIndex.from_tuples(zip(level_i0, level_i1))
df1.columns = multi_index_c
df1.index = multi_index_i

# 不要な行・列の削除
df1 = df1.drop(index=df.index[[0, 1]], columns=df.columns[[0, 1]])

# インデックス、カラムのnames属性の設定
df1.index.names = ['地域コード', '都道府県']
df1.columns.names = ['項目コード', '項目年次', '項目名'] 
# output
df1.head()

,項目コード,A1101,A1102,A1301,A1302,A1303,A1700,A4101,A4103,A4200,A5101,...,I5103,I6100,I6200,I6300,J2503,J2506,L3221,L322101,L322102,L322109
,項目年次,2021,2021,2021,2021,2021,2020,2020,2019,2020,2021,...,2020,2020,2020,2020,2020,2020,2021,2021,2021,2021
,項目名,総人口,日本人人口,15歳未満人口,15～64歳人口,65歳以上人口,外国人人口,出生数,合計特殊出生率,死亡数,転入者数（日本人移動者）,...,歯科診療所数,医師数,歯科医師数,薬剤師数,保育所等数,保育所等在所児数,消費支出（二人以上の世帯）,食料費（二人以上の世帯）,住居費（二人以上の世帯）,教養娯楽費（二人以上の世帯）
地域コード,都道府県,,,,,,,,,,,,,,,,,,,,,
R00000,全国,125502000,122780000,14784000,74504000,36214000,2402460,840835,1.36,1372755,2242845,...,67874,339623,107443,321982,29474,2624335,279024,75761,18329,24545
R01000,北海道,5183000,5147000,544000,2953000,1686000,34321,29523,1.24,65078,48832,...,2840,13731,4418,11802,1045,79404,268396,71448,22013,23762
R02000,青森県,1221000,1216000,127000,676000,419000,5409,6837,1.38,17905,16169,...,511,2773,735,2345,475,30723,245117,69503,13232,19092
R03000,岩手県,1196000,1189000,129000,659000,409000,6937,6718,1.35,17204,16149,...,566,2700,1016,2536,390,28895,272911,73738,17181,22302
R04000,宮城県,2290000,2269000,264000,1371000,655000,19453,14480,1.23,24632,44079,...,1052,5950,1896,5502,492,39497,284171,75689,19852,24985


## 解答2

In [3]:
# DataFrameのコンストラクタを使用したMultiIndexの設定　
df2 = df

# MultiIndexの設定
df2 = pd.DataFrame(df2.values, index= [level_i0, level_i1], columns= [level_c0, level_c1, level_c2])

# 不要な行・列の削除
df2 = df2.drop(index=df2.index[[0, 1]], columns=df2.columns[[0, 1]])

# インデックス、カラムのnames属性の設定
df2.index.names = ['地域コード', '都道府県']
df2.columns.names = ['項目コード', '項目年次', '項目名'] 
# output
df2.head()

,項目コード,A1101,A1102,A1301,A1302,A1303,A1700,A4101,A4103,A4200,A5101,...,I5103,I6100,I6200,I6300,J2503,J2506,L3221,L322101,L322102,L322109
,項目年次,2021,2021,2021,2021,2021,2020,2020,2019,2020,2021,...,2020,2020,2020,2020,2020,2020,2021,2021,2021,2021
,項目名,総人口,日本人人口,15歳未満人口,15～64歳人口,65歳以上人口,外国人人口,出生数,合計特殊出生率,死亡数,転入者数（日本人移動者）,...,歯科診療所数,医師数,歯科医師数,薬剤師数,保育所等数,保育所等在所児数,消費支出（二人以上の世帯）,食料費（二人以上の世帯）,住居費（二人以上の世帯）,教養娯楽費（二人以上の世帯）
地域コード,都道府県,,,,,,,,,,,,,,,,,,,,,
R00000,全国,125502000,122780000,14784000,74504000,36214000,2402460,840835,1.36,1372755,2242845,...,67874,339623,107443,321982,29474,2624335,279024,75761,18329,24545
R01000,北海道,5183000,5147000,544000,2953000,1686000,34321,29523,1.24,65078,48832,...,2840,13731,4418,11802,1045,79404,268396,71448,22013,23762
R02000,青森県,1221000,1216000,127000,676000,419000,5409,6837,1.38,17905,16169,...,511,2773,735,2345,475,30723,245117,69503,13232,19092
R03000,岩手県,1196000,1189000,129000,659000,409000,6937,6718,1.35,17204,16149,...,566,2700,1016,2536,390,28895,272911,73738,17181,22302
R04000,宮城県,2290000,2269000,264000,1371000,655000,19453,14480,1.23,24632,44079,...,1052,5950,1896,5502,492,39497,284171,75689,19852,24985


## 解答3

In [4]:
# MultiIndexのレベルの交換とソート
df3 = df2

df3 = df3.swaplevel(i=0, j=1, axis=1).sort_index(axis=1)
# Output
df3.head()

項目年次            2016                                              \
項目コード          C2108       C210836       C210837         C210839   
項目名         事業所数（民営） 事業所数（民営）（建設業） 事業所数（民営）（製造業） 事業所数（民営）（情報通信業）   
地域コード  都道府県                                                        
R00000 全国    5340783        492734        454800           63574   
R01000 北海道    224718         21361         10818            2206   
R02000 青森県     58116          5750          3002             366   
R03000 岩手県     58415          5677          3884             432   
R04000 宮城県     97974         10640          5350            1036   

項目年次                                                                         \
項目コード                 C210841               C210847                 C210848   
項目名         事業所数（民営）（卸売業，小売業） 事業所数（民営）（宿泊業，飲食サービス業） 事業所数（民営）（生活関連サービス業，娯楽業）   
地域コード  都道府県                                                                   
R00000 全国             1355060                696396                  470713   
R01000 北海道              56213                 32259                   20386   
R02000 青森県              15799                  8052                    6494   
R03000 岩手県              15404                  7095                    6016   
R04000 宮城県              27102                 11621                    8736   

項目年次                                                 ...  2021          \
項目コード               C210850     C2208       C220836  ... E6102   E6301   
項目名         事業所数（民営）（医療，福祉）  従業者数（民営） 従業者数（民営）（建設業）  ...   大学数 短期大学学生数   
地域コード  都道府県                                          ...                 
R00000 全国            429173  56872826       3690740  ...   803   99416   
R01000 北海道            18506   2165925        182143  ...    37    4069   
R02000 青森県             4899    498988         50820  ...    10     950   
R03000 岩手県             4772    525264         50849  ...     6     726   
R04000 宮城県             7573   1006886         99631  ...    14    2701   

項目年次                                                                           \
項目コード          E6302    G5105      G7101     G7102         L3221      L322101   
項目名            大学学生数 一般旅券発行件数     延べ宿泊者数 外国人延べ宿泊者数 消費支出（二人以上の世帯） 食料費（二人以上の世帯）   
地域コード  都道府県                                                                     
R00000 全国    2625688   513943  258876780   3438420        279024        75761   
R01000 北海道     79729    10839   15812120     46380        268396        71448   
R02000 青森県     15419     1519    2788990     13240        245117        69503   
R03000 岩手県     11340     1515    3660190     11470        272911        73738   
R04000 宮城県     49580     5053    5631070     35570        284171        75689   

項目年次                                     
項目コード            L322102        L322109  
項目名         住居費（二人以上の世帯） 教養娯楽費（二人以上の世帯）  
地域コード  都道府県                              
R00000 全国          18329          24545  
R01000 北海道         22013          23762  
R02000 青森県         13232          19092  
R03000 岩手県         17181          22302  
R04000 宮城県         19852          24985  

[5 rows x 90 columns]

## 解答4

In [5]:
# stackメソッドによる指定columnsのindexへの転換
df4 = df2

df4 = df4.stack([0,2])
# (Output)
display(df4)

# 行列全体の変換
df4 = df4.T
# Output
df4.head()

項目年次                               2016 2018 2019   2020       2021
地域コード  都道府県 項目コード   項目名                                            
R00000 全国   A1101   総人口             NaN  NaN  NaN    NaN  125502000
            A1102   日本人人口           NaN  NaN  NaN    NaN  122780000
            A1301   15歳未満人口         NaN  NaN  NaN    NaN   14784000
            A1302   15～64歳人口        NaN  NaN  NaN    NaN   74504000
            A1303   65歳以上人口         NaN  NaN  NaN    NaN   36214000
...                                 ...  ...  ...    ...        ...
R47000 沖縄県  J2506   保育所等在所児数        NaN  NaN  NaN  54795        NaN
            L3221   消費支出（二人以上の世帯）   NaN  NaN  NaN    NaN     228284
            L322101 食料費（二人以上の世帯）    NaN  NaN  NaN    NaN      66970
            L322102 住居費（二人以上の世帯）    NaN  NaN  NaN    NaN      27286
            L322109 教養娯楽費（二人以上の世帯）  NaN  NaN  NaN    NaN      17141

[4320 rows x 5 columns]

地域コード     R00000                                                            \
都道府県          全国                                                             
項目コード      A1101      A1102     A1301     A1302     A1303    A1700   A4101   
項目名          総人口      日本人人口   15歳未満人口  15～64歳人口   65歳以上人口    外国人人口     出生数   
項目年次                                                                         
2016         NaN        NaN       NaN       NaN       NaN      NaN     NaN   
2018         NaN        NaN       NaN       NaN       NaN      NaN     NaN   
2019         NaN        NaN       NaN       NaN       NaN      NaN     NaN   
2020         NaN        NaN       NaN       NaN       NaN  2402460  840835   
2021   125502000  122780000  14784000  74504000  36214000      NaN     NaN   

地域コード                                ... R47000                          \
都道府県                                 ...    沖縄県                           
項目コード   A4103    A4200        A5101  ...  I5103 I6100 I6200 I6300 J2503   
項目名   合計特殊出生率      死亡数 転入者数（日本人移動者）  ... 歯科診療所数   医師数 歯科医師数  薬剤師数 保育所等数   
項目年次                                 ...                                  
2016      NaN      NaN          NaN  ...    NaN   NaN   NaN   NaN   NaN   
2018      NaN      NaN          NaN  ...    NaN   NaN   NaN   NaN   NaN   
2019     1.36      NaN          NaN  ...    NaN   NaN   NaN   NaN   NaN   
2020      NaN  1372755          NaN  ...    607  3887   885  2432   606   
2021      NaN      NaN      2242845  ...    NaN   NaN   NaN   NaN   NaN   

地域コード                                                                  
都道府県                                                                   
項目コード    J2506         L3221      L322101      L322102        L322109  
項目名   保育所等在所児数 消費支出（二人以上の世帯） 食料費（二人以上の世帯） 住居費（二人以上の世帯） 教養娯楽費（二人以上の世帯）  
項目年次                                                                   
2016       NaN           NaN          NaN          NaN            NaN  
2018       NaN           NaN          NaN          NaN            NaN  
2019       NaN           NaN          NaN          NaN            NaN  
2020     54795           NaN          NaN          NaN            NaN  
2021       NaN        228284        66970        27286          17141  

[5 rows x 4320 columns]

## 解答5

* MultiIndexは、

In [6]:
# MultiIndesからのデータ抽出
df5 = df3

# slice()関数
df5 = df5.loc[(slice(None),slice(None)), (2020, slice('I510120', 'I6300'), slice(None) )]
# (Output)
display(df5.head())

# IndexSliceクラス
df5 = df5.loc[pd.IndexSlice[:, :], pd.IndexSlice[2020, 'I510120': 'I6300', :]]
# Output
df5.head()

項目年次           2020                                       
項目コード       I510120   I5102  I5103   I6100   I6200   I6300
項目名           一般病院数  一般診療所数 歯科診療所数     医師数   歯科医師数    薬剤師数
地域コード  都道府県                                               
R00000 全国      7179  102612  67874  339623  107443  321982
R01000 北海道      479    3351   2840   13731    4418   11802
R02000 青森県       77     862    511    2773     735    2345
R03000 岩手県       77     877    566    2700    1016    2536
R04000 宮城県      109    1686   1052    5950    1896    5502

項目年次           2020                                       
項目コード       I510120   I5102  I5103   I6100   I6200   I6300
項目名           一般病院数  一般診療所数 歯科診療所数     医師数   歯科医師数    薬剤師数
地域コード  都道府県                                               
R00000 全国      7179  102612  67874  339623  107443  321982
R01000 北海道      479    3351   2840   13731    4418   11802
R02000 青森県       77     862    511    2773     735    2345
R03000 岩手県       77     877    566    2700    1016    2536
R04000 宮城県      109    1686   1052    5950    1896    5502

## 問題6

* MultiIndexの操作を効率的に行う工夫として、今回のデータセットのように対応するコードが指定できる場合は検索機能を付けてやることが有益です。

In [7]:
# 「項目コード」を指定して検索＆抽出
df6 = df3

# 検索key
item_key = 'A'

# columnsのMultiIndexから「項目コード」のlistを抽出し、
# filterメソッドを使用してコードの先頭文字列がitem_keyに合致するlistを生成
item_code = [i for i in filter(lambda x: x[0:len(item_key)] == item_key ,
                               [code for _, code, _ in df6.columns])]

# IndexSliceクラスを使用して、item_codeで該当する「項目コード」のデータを抽出
df6 = df6.loc[pd.IndexSlice[:, :],pd.IndexSlice[:, item_code, :]] 

#Output
df6.head()

項目年次           2019     2020                                                  \
項目コード         A4103    A1700   A4101    A4200   A710101    A710201   A810105   
項目名         合計特殊出生率    外国人人口     出生数      死亡数     一般世帯数    一般世帯人員数     単独世帯数   
地域コード  都道府県                                                                    
R00000 全国      1.36  2402460  840835  1372755  55704949  123162995  21151042   
R01000 北海道     1.24    34321   29523    65078   2469063    5032739    999825   
R02000 青森県     1.38     5409    6837    17905    509649    1191534    168917   
R03000 岩手県     1.35     6937    6718    17204    490828    1174106    163290   
R04000 宮城県     1.23    19453   14480    24632    980549    2251390    362255   

項目年次                              2021                                 \
項目コード         A9101   A9201      A1101      A1102     A1301     A1302   
項目名            婚姻件数    離婚件数        総人口      日本人人口   15歳未満人口  15～64歳人口   
地域コード  都道府県                                                             
R00000 全国    525507  193253  125502000  122780000  14784000  74504000   
R01000 北海道    20904    9070    5183000    5147000    544000   2953000   
R02000 青森県     4032    1915    1221000    1216000    127000    676000   
R03000 岩手県     3918    1679    1196000    1189000    129000    659000   
R04000 宮城県     8921    3553    2290000    2269000    264000   1371000   

項目年次                                             
項目コード           A1303        A5101        A5102  
項目名           65歳以上人口 転入者数（日本人移動者） 転出者数（日本人移動者）  
地域コード  都道府県                                      
R00000 全国    36214000      2242845      2242845  
R01000 北海道    1686000        48832        50979  
R02000 青森県     419000        16169        20396  
R03000 岩手県     409000        16149        19022  
R04000 宮城県     655000        44079        43962

### 問題7

In [8]:
# MultiIndexの列の追加
df7 = df5

df7[(2020,'I9000','病院・診療所当り医師数')] = df7[(2020,'I6100','医師数')] / \
                                              (df7[(2020,'I510120','一般病院数')] + df7[(2020,'I5102','一般診療所数')])
df7[(2020,'I9100','歯科診療所当り歯科医師数')] = df7[(2020,'I6200','歯科医師数')] / df7[(2020,'I5103','歯科診療所数')]
# Output
df7.head()

項目年次           2020                                                     \
項目コード       I510120   I5102  I5103   I6100   I6200   I6300       I9000   
項目名           一般病院数  一般診療所数 歯科診療所数     医師数   歯科医師数    薬剤師数 病院・診療所当り医師数   
地域コード  都道府県                                                              
R00000 全国      7179  102612  67874  339623  107443  321982    3.093359   
R01000 北海道      479    3351   2840   13731    4418   11802    3.585117   
R02000 青森県       77     862    511    2773     735    2345    2.953142   
R03000 岩手県       77     877    566    2700    1016    2536    2.830189   
R04000 宮城県      109    1686   1052    5950    1896    5502    3.314763   

項目年次                      
項目コード              I9100  
項目名         歯科診療所当り歯科医師数  
地域コード  都道府県               
R00000 全国       1.582977  
R01000 北海道      1.555634  
R02000 青森県      1.438356  
R03000 岩手県      1.795053  
R04000 宮城県      1.802281

## 問題8

* 複数条件で抽出するときは & や | を使用すること、および各々の条件式を () で括ることがポイントになります。

In [9]:
# MultiIndexからの複数条件による抽出
df8 = df7

# 全国平均病院・診療所当り医師数
mean_hospital = df8.loc[('R00000', '全国'), (2020, 'I9000', '病院・診療所当り医師数')] 
# 全国平均歯科医院当り歯科医師数
mean_clinic = df8.loc[('R00000', '全国'), (2020, 'I9100', '歯科診療所当り歯科医師数')] 

df8 = df8[(df8[(2020, 'I9000', '病院・診療所当り医師数')] > mean_hospital) \
      & (df8[(2020, 'I9100', '歯科診療所当り歯科医師数')] > mean_clinic)].index
# Output
df8

MultiIndex([('R04000', '宮城県'),
            ('R12000', '千葉県'),
            ('R13000', '東京都'),
            ('R20000', '長野県'),
            ('R23000', '愛知県'),
            ('R33000', '岡山県'),
            ('R36000', '徳島県'),
            ('R40000', '福岡県'),
            ('R43000', '熊本県')],
           names=['地域コード', '都道府県'])

## 問題9

* reset_indexメソッドの引数にdrop=Trueを指定することで不要なindexを削除することができます。
* この例ではNaNの処理も必要です。

In [10]:
# MUltiIndexからのデータ抽出と次元の削減
df9 = df3

# 大学関連データの抽出
df9 = df9.loc[pd.IndexSlice['R01000':, :],pd.IndexSlice[:, 'E6102':'E6302', :]]
# (Output)
display(df9.head())
# 次元の削減
df9 = df9.stack([0, 1]).reset_index(level=['地域コード', '項目コード','項目年次'], drop=True)
# (Output)
display(df9.head())
df9 = df9.groupby('都道府県').sum()
# (Output)
display(df9.head())

項目年次         2021               
項目コード       E6102   E6301  E6302
項目名           大学数 短期大学学生数  大学学生数
地域コード  都道府県                     
R01000 北海道     37    4069  79729
R02000 青森県     10     950  15419
R03000 岩手県      6     726  11340
R04000 宮城県     14    2701  49580
R05000 秋田県      7     577   8904

項目名,大学学生数,大学数,短期大学学生数
都道府県,,,
北海道,NaN,37,NaN
北海道,NaN,NaN,4069
北海道,79729,NaN,NaN
青森県,NaN,10,NaN
青森県,NaN,NaN,950


項目名,大学学生数,大学数,短期大学学生数
都道府県,,,
三重県,14062,7,1419
京都府,143095,34,3103
佐賀県,7776,2,811
兵庫県,115536,36,5439
北海道,79729,37,4069


## 問題10

* この例というよりも一般的にはデータが大量にある場合の情報量削減の方法のひとつとしてビニングがあります。
* category型という特殊な型を使用するので、初めての場合はこの機会に深堀してみると面白いと思います。

In [11]:
# ビニングと度数分布表

df10 = df9
df10['大学数'] = pd.cut(df10['大学数'],[0, 20, 40, 60, 80, 100, 120, 140, 160])
df10['大学学生数'] = pd.cut(df10['大学学生数'], 5)
df10['短期大学学生数'] = pd.qcut(df10['短期大学学生数'], 5)

# (Output)
display(df10.head())
print(df10.groupby('大学数')['大学数'].count())
print(df10.groupby('大学学生数')['大学学生数'].count())
print(df10.groupby('短期大学学生数')['短期大学学生数'].count())


項目名,大学学生数,大学数,短期大学学生数
都道府県,,,
三重県,"(6050.757, 140769.6]","(0, 20]","(977.2, 1536.2]"
京都府,"(140769.6, 274818.2]","(20, 40]","(3093.2, 9688.0]"
佐賀県,"(6050.757, 140769.6]","(0, 20]","(718.8, 977.2]"
兵庫県,"(6050.757, 140769.6]","(20, 40]","(3093.2, 9688.0]"
北海道,"(6050.757, 140769.6]","(20, 40]","(3093.2, 9688.0]"


大学数
(0, 20]       35
(20, 40]       9
(40, 60]       2
(60, 80]       0
(80, 100]      0
(100, 120]     0
(120, 140]     0
(140, 160]     1
Name: 大学数, dtype: int64
大学学生数
(6050.757, 140769.6]    42
(140769.6, 274818.2]     4
(274818.2, 408866.8]     0
(408866.8, 542915.4]     0
(542915.4, 676964.0]     1
Name: 大学学生数, dtype: int64
短期大学学生数
(166.999, 718.8]    10
(718.8, 977.2]       9
(977.2, 1536.2]      9
(1536.2, 3093.2]     9
(3093.2, 9688.0]    10
Name: 短期大学学生数, dtype: int64
